# TriageFlow: Incident Triage Demo

Multi-agent incident triage with Domino GenAI tracing.

## Setup

Save your API key as a Domino user environment variable:
1. **Account Settings** → **User Environment Variables**
2. Add `OPENAI_API_KEY` or `ANTHROPIC_API_KEY`

In [17]:
import os
import yaml
import pandas as pd
import ipywidgets as widgets
from IPython.display import display

## Select Provider

Choose your LLM provider. Both use the same pipeline with provider-specific auto-tracing.

In [18]:
provider_dropdown = widgets.Dropdown(
    options=["openai", "anthropic"],
    value="openai",
    description="Provider:"
)
display(provider_dropdown)

Dropdown(description='Provider:', options=('openai', 'anthropic'), value='openai')

## Load Configuration

All prompts, model settings, and agent parameters are centralized in `config.yaml`.

In [19]:
with open("config.yaml") as f:
    config = yaml.safe_load(f)

provider = provider_dropdown.value
model = config["models"][provider]
print(f"Provider: {provider}\nModel: {model}")

Provider: anthropic
Model: claude-sonnet-4-20250514


## Initialize Client & Auto-Tracing

MLflow's `autolog()` automatically captures all LLM calls without additional instrumentation.

In [20]:
import mlflow

if provider == "openai":
    from openai import OpenAI
    client = OpenAI()
    mlflow.openai.autolog()
else:
    from anthropic import Anthropic
    client = Anthropic()
    mlflow.anthropic.autolog()

print(f"Auto-tracing enabled for {provider}")

Auto-tracing enabled for anthropic


## Import Domino Tracing

- `add_tracing`: Decorator for capturing inputs, outputs, and evaluation metrics
- `DominoRun`: Context manager for aggregating metrics across multiple traces

In [21]:
from domino.agents.tracing import add_tracing
from domino.agents.logging import DominoRun

In [22]:
from models import Incident, IncidentSource
from agents import classify_incident, assess_impact, match_resources, draft_response

## Define Inline Evaluators

Evaluators run automatically after each agent call, computing quality metrics from inputs and outputs. Results are attached directly to the trace.

In [ ]:
def classifier_evaluator(span) -> dict:
    """Check if urgency aligns with critical keywords."""
    inputs = span.inputs or {}
    outputs = span.outputs
    # Handle Pydantic model output
    if hasattr(outputs, "model_dump"):
        outputs = outputs.model_dump()
    outputs = outputs or {}
    
    urgency = outputs.get("urgency", 3)
    confidence = outputs.get("confidence", 0.5)
    description = str(inputs.get("incident", {}).get("description", "")).lower()
    critical_keywords = ["breach", "outage", "critical", "emergency", "down", "compromised"]
    has_critical = any(kw in description for kw in critical_keywords)
    urgency_appropriate = "appropriate" if (has_critical and urgency >= 4) or (not has_critical and urgency < 4) else "misaligned"
    return {
        "classification_confidence": confidence,
        "urgency_appropriate": urgency_appropriate,
        "category_assigned": outputs.get("category", "unknown")
    }

In [ ]:
def impact_evaluator(span) -> dict:
    """Verify impact score correlates with urgency."""
    inputs = span.inputs or {}
    outputs = span.outputs
    if hasattr(outputs, "model_dump"):
        outputs = outputs.model_dump()
    outputs = outputs or {}
    
    impact_score = outputs.get("impact_score", 5.0)
    classification = inputs.get("classification", {})
    if hasattr(classification, "model_dump"):
        classification = classification.model_dump()
    urgency = classification.get("urgency", 3) if isinstance(classification, dict) else 3
    similar_count = len(outputs.get("similar_incidents", []))
    impact_urgency_aligned = abs(impact_score / 2 - urgency) <= 1.5
    return {
        "impact_score": impact_score,
        "historical_matches_found": similar_count,
        "impact_urgency_alignment": "aligned" if impact_urgency_aligned else "misaligned",
        "blast_radius": outputs.get("blast_radius", "unknown")
    }

In [ ]:
def resource_evaluator(span) -> dict:
    """Evaluate resource match quality and SLA compliance."""
    outputs = span.outputs
    if hasattr(outputs, "model_dump"):
        outputs = outputs.model_dump()
    outputs = outputs or {}
    
    primary = outputs.get("primary_responder", {})
    if hasattr(primary, "model_dump"):
        primary = primary.model_dump()
    match_score = primary.get("match_score", 0.5) if isinstance(primary, dict) else 0.5
    sla_met = outputs.get("sla_met", False)
    backups = outputs.get("backup_responders", [])
    return {
        "resource_match_score": match_score,
        "sla_compliance": "met" if sla_met else "at_risk",
        "backup_count": len(backups) if isinstance(backups, list) else 0
    }

In [ ]:
def response_evaluator(span) -> dict:
    """Assess response plan completeness."""
    outputs = span.outputs
    if hasattr(outputs, "model_dump"):
        outputs = outputs.model_dump()
    outputs = outputs or {}
    
    communications = outputs.get("communications", [])
    action_items = outputs.get("action_items", [])
    completeness = outputs.get("completeness_score", 0.5)
    return {
        "completeness_score": completeness,
        "communication_count": len(communications) if isinstance(communications, list) else 0,
        "action_item_count": len(action_items) if isinstance(action_items, list) else 0
    }

## Define Traced Pipeline Functions

The `@add_tracing` decorator wraps each agent call, capturing:
- Function inputs and outputs
- Nested LLM calls (via `autolog_frameworks`)
- Evaluation metrics (via `evaluator`)

In [27]:
framework = [provider]

@add_tracing(name="classify_incident", autolog_frameworks=framework, evaluator=classifier_evaluator)
def traced_classify(incident: Incident):
    return classify_incident(client, provider, model, incident, config)

@add_tracing(name="assess_impact", autolog_frameworks=framework, evaluator=impact_evaluator)
def traced_assess(incident, classification):
    return assess_impact(client, provider, model, incident, classification, config)

@add_tracing(name="match_resources", autolog_frameworks=framework, evaluator=resource_evaluator)
def traced_match(classification, impact):
    return match_resources(client, provider, model, classification, impact, config)

@add_tracing(name="draft_response", autolog_frameworks=framework, evaluator=response_evaluator)
def traced_draft(incident, classification, impact, resources):
    return draft_response(client, provider, model, incident, classification, impact, resources, config)

## Load Sample Incidents

Eight incidents spanning financial services, healthcare, energy, and public sector verticals.

In [29]:
df = pd.read_csv("example-data/sample_incidents.csv")
df

,ticket_id,description,source,reporter,affected_system,initial_severity
0,INC-2024-2001,Multiple users reporting inability to access t...,user_report,Help Desk,Trading Platform,4
1,INC-2024-2002,Automated security scan detected unusual outbo...,automated_scan,NaN,DB-PROD-03,5
2,INC-2024-2003,Monthly compliance report generation failed. E...,monitoring,NaN,Compliance Reporting System,3
3,INC-2024-2004,User john.smith@company.com unable to reset pa...,user_report,john.smith@company.com,Identity Management,2
4,INC-2024-2005,Patient data synchronization between EMR and b...,monitoring,NaN,EMR-Billing Integration,4
5,INC-2024-2006,SCADA monitoring system showing intermittent c...,monitoring,NaN,SCADA Control System,5
6,INC-2024-2007,Citizen services portal experiencing slow resp...,user_report,Social Media Team,Citizen Portal,3
7,INC-2024-2008,Database replication lag detected between prim...,monitoring,NaN,Database Cluster,4


In [30]:
def row_to_incident(row) -> Incident:
    return Incident(
        ticket_id=row["ticket_id"],
        description=row["description"],
        source=IncidentSource(row["source"]),
        reporter=row["reporter"] if pd.notna(row["reporter"]) else None,
        affected_system=row["affected_system"] if pd.notna(row["affected_system"]) else None,
        initial_severity=int(row["initial_severity"]) if pd.notna(row["initial_severity"]) else None
    )

incidents = [row_to_incident(row) for _, row in df.iterrows()]
print(f"Loaded {len(incidents)} incidents")

Loaded 8 incidents


## Run Triage Pipeline

`DominoRun` aggregates metrics across all traces in the batch via `custom_summary_metrics`. Supported aggregations: `mean`, `median`, `stdev`, `min`, `max`.

In [31]:
aggregated_metrics = [
    ("classification_confidence", "mean"),
    ("classification_confidence", "stdev"),
    ("impact_score", "median"),
    ("impact_score", "max"),
    ("resource_match_score", "mean"),
    ("completeness_score", "mean"),
]

In [32]:
results = []

with DominoRun(custom_summary_metrics=aggregated_metrics) as run:
    for incident in incidents:
        print(f"Processing {incident.ticket_id}...")
        
        classification = traced_classify(incident)
        impact = traced_assess(incident, classification)
        resources = traced_match(classification, impact)
        response = traced_draft(incident, classification, impact, resources)
        
        results.append({
            "ticket_id": incident.ticket_id,
            "classification": classification,
            "impact": impact,
            "resources": resources,
            "response": response
        })
        print(f"  → {classification.category.value} | Urgency: {classification.urgency} | Impact: {impact.impact_score}")

print(f"\nProcessed {len(results)} incidents")

Processing INC-2024-2001...
🏃 View run clean-lamb-343 at: http://127.0.0.1:8768/#/experiments/1993/runs/f321557caeb54f568265455be6db24ad
🧪 View experiment at: http://127.0.0.1:8768/#/experiments/1993


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

Trace(trace_id=tr-fd1edc73c9476db8d32de0e97aee8e65)

## Results Summary

In [ ]:
summary = pd.DataFrame([{
    "Ticket": r["ticket_id"],
    "Category": r["classification"].category.value,
    "Urgency": r["classification"].urgency,
    "Impact": r["impact"].impact_score,
    "Responder": r["resources"].primary_responder.name,
    "SLA Met": r["resources"].sla_met
} for r in results])
summary

## Sample Communication

Each incident generates tailored communications for technical teams, management, and affected users.

In [ ]:
sample = results[0]
print(f"Ticket: {sample['ticket_id']}\n")
for comm in sample["response"].communications:
    print(f"--- {comm.audience.upper()} ---")
    print(f"Subject: {comm.subject}")
    print(f"{comm.body[:300]}...\n")

## Next Steps

Open **Domino Experiment Manager** to view:
- Execution flow across all 4 agents
- Inline evaluation metrics per trace
- Aggregated statistics across the batch